# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-10 04:58:13] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-10 04:58:13] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-10 04:58:13] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-10 04:58:16] INFO server_args.py:1638: Attention backend not specified. Use fa3 backend by default.


[2026-01-10 04:58:16] INFO server_args.py:2535: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=96 avail_mem=74.67 GB):  15%|█▌        | 3/20 [00:00<00:01,  8.56it/s] 

Capturing batches (bs=56 avail_mem=74.65 GB):  40%|████      | 8/20 [00:00<00:00, 15.20it/s]

Capturing batches (bs=16 avail_mem=74.63 GB):  65%|██████▌   | 13/20 [00:00<00:00, 17.79it/s]

Capturing batches (bs=1 avail_mem=74.57 GB):  90%|█████████ | 18/20 [00:01<00:00, 15.91it/s]

Capturing batches (bs=1 avail_mem=74.57 GB): 100%|██████████| 20/20 [00:01<00:00, 14.58it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Selena. I have two goals for this year: 1. I want to continue my high school school after my high school education. 2. I want to get into a college I have always wanted to attend. I am a freshman at a university. The school has two campuses. I have to pick one based on how I feel I should go and on what I will have to bring with me. I know this will be hard and confusing. I need help with this. How can I make this decision?

### Recommendation

---

**Step 1: Understand Your Values**
Before deciding on a college, it's important to
Prompt: The president of the United States is
Generated text:  a citizen of which country?
A: China
B: Canada
C: France
D: Japan
To determine which country the president of the United States is a citizen of, let's analyze the information step by step.

1. **Understanding the President of the United States:**
   - The president of the United States is the leader of the federal government of the United States.
   - The

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [insert a few key points about your background, experience, or personality]. I'm always looking for new challenges and opportunities to grow and learn. What do you like to do for fun? I love to read, travel, and spend time with my family. What's your favorite hobby? I love to play guitar and spend time with my friends. What's your favorite book? I love to read books about animals and nature. What's your

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. It is also home to the French Parliament and the French National Museum. Paris is a bustling metropolis with a rich cultural heritage and is a major tourist destination. The city is known for its cuisine, fashion, and art scene. It is also home to the French Parliament and the French National Museum. Paris is a major tourist destination and is known for its iconic landmarks, cultural heritage, and cuisine. It is also home to the French Parliament and the French National Museum. Paris is a major tourist destination and is known

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction. Here are some of the most likely trends that are expected to shape the future of AI:

1. Increased Use of AI in Healthcare: AI is expected to play a significant role in healthcare in the coming years. AI-powered diagnostic tools, personalized treatment plans, and predictive analytics are expected to revolutionize the way we diagnose and treat diseases.

2. Enhanced Personalization: AI will enable personalized medicine, where the treatment plan is tailored to an individual's specific needs and characteristics. This will lead to better outcomes and reduced costs.

3. Increased Use of



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name]. I am a versatile and experienced chef from [Location], specializing in [Your specialty]. I've been in the culinary world for [Number] years, and I'm a passionate advocate for using fresh, seasonal ingredients to create delicious, nutritious meals for my customers. I'm excited to help you discover new recipes, fresh produce, and tasty treats. What brings you to our establishment? Let's make memories together and create unforgettable experiences for our customers! [Your Name] [Your Profession] [Your Background] [Your Interests and Skills] [Your Passion for Culinary Arts] [Your Unique Selling Points]
Hello, my

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, known as the "City of Love" due to its iconic status and cultural importance. It is a vibrant metropolis with a rich history and diverse popul

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Character

 Name

],

...

 advantage

 of

 their

 individual

 learning

In [6]:
llm.shutdown()